In [2]:
import json
import pandas as pd
import os

In [3]:
# category = "["American Football","Aquatics","Archery","Athletics","Australian Rules Football","Auto Racing","Badminton","Baseball and Softball","Basketball","Biathlon","Billiards","Bobsleigh and Skeleton","Bowling","Boxing","Canadian Football","Canoeing","Chess and Cards","Cricket","Curling","Cycling","DanceSport","Darts","Dragon Boat","Equestrianism","eSports","Fencing","Field Hockey","Football","Gaelic Football","Frisbee","Golf","Gymnastics","Handball","Ice Hockey","Judo","Kabaddi","Karate","Kickboxing","Lacrosse","Luge","MMA","Modern Pentathlon","Motorcycle Sports","Muaythai","Other Sport","Powerboating","Racquetball","Roller Sports","Rowing","Rugby League","Rugby Union","Sailing","Sanda","Shooting","Skating","Ski Mountaineering","Skiing","Sport Climbing","Squash","Surfing","Table Tennis","Taekwondo","Tennis","Triathlon","Volleyball","Weightlifting","Wrestling","Wushu"]"

# prompt = "You are an expert in text classification with a specialization in the sports domain. Please follow these specific instructions: 1. Include the sport types for the competition. The sport types must belong in the following categories: ["American Football", "Aquatics", "Archery", "Athletics", "Australian Rules Football", "Auto Racing", "Badminton", "Baseball and Softball", "Basketball", "Biathlon", "Billiards", "Bobsleigh and Skeleton", "Bowling", "Boxing", "Canadian Football", "Canoeing", "Chess and Cards", "Cricket", "Curling", "Cycling", "DanceSport", "Darts", "Dragon Boat", "Equestrianism", "eSports", "Fencing", "Field Hockey", "Football", "Gaelic Football", "Frisbee", "Golf", "Gymnastics", "Handball", "Ice Hockey", "Judo", "Kabaddi", "Karate", "Kickboxing", "Lacrosse", "Luge", "MMA", "Modern Pentathlon", "Motorcycle Sports", "Muaythai", "Other Sport", "Powerboating", "Racquetball", "Roller Sports", "Rowing", "Rugby League", "Rugby Union", "Sailing", "Sanda", "Shooting", "Skating", "Ski Mountaineering", "Skiing", "Sport Climbing", "Squash", "Surfing", "Table Tennis", "Taekwondo", "Tennis", "Triathlon", "Volleyball", "Weightlifting", "Wrestling", "Wushu"]. Give all possible sport types if the text is not clear. However, if the text is clear and can be classified into a certain sport type, only give one type. For example, "NORWAY VS DENMARK NORWAY VS DENMARK" can be "Football, Basketball, Volleyball". 2. Each result should be a string object indicating the sport types. For example: "Football" or "Football, Basketball". 3. Only give final string object. I will save them in python."
category = ["Football", "Basketball", "Volleyball", "Table Tennis", "Tennis", "Badminton", "Others"]
# '["American Football", "Aquatics", "Archery", "Athletics", "Australian Rules Football", "Auto Racing", "Badminton", "Baseball and Softball", "Basketball", "Biathlon", "Billiards", "Bobsleigh and Skeleton", "Bowling", "Boxing", "Canadian Football", "Canoeing", "Chess and Cards", "Cricket", "Curling", "Cycling", "DanceSport", "Darts", "Dragon Boat", "Equestrianism", "eSports", "Fencing", "Field Hockey", "Football", "Gaelic Football", "Frisbee", "Golf", "Gymnastics", "Handball", "Ice Hockey", "Judo", "Kabaddi", "Karate", "Kickboxing", "Lacrosse", "Luge", "MMA", "Modern Pentathlon", "Motorcycle Sports", "Muaythai", "Other Sport", "Powerboating", "Racquetball", "Roller Sports", "Rowing", "Rugby League", "Rugby Union", "Sailing", "Sanda", "Shooting", "Skating", "Ski Mountaineering", "Skiing", "Sport Climbing", "Squash", "Surfing", "Table Tennis", "Taekwondo", "Tennis", "Triathlon", "Volleyball", "Weightlifting", "Wrestling", "Wushu"]'
prompt = '''
You are an expert in text classification with a specialization in the sports domain. Please classify the sport types for the given context. Note that return only the final string object.
'''

In [6]:
prompt = '''You are an expert in text classification with a specialization in the sports domain. Please follow these specific instructions:
1. Include the sport types for the competition: The sport types must belong to the given categories. 
- For unclear texts: List all possible sport types that match the text (e.g., "Football, Basketball, Volleyball").
- For clear cases: When the text clearly identifies a single sport, return only that specific sport type (e.g., "Football").
2. Output format:
- Return only the final string object.
- For clear text like "UEFA Champions League": "Football"
- For unclear text like "NORWAY VS DENMARK": "Football, Basketball, Volleyball"
3. Only provide the final string output, as I will save it in Python.'''

In [26]:
prompt = '''
您是文本分类方面的专家，专攻体育领域。请按照以下具体说明操作：
1. 给出比赛的运动类型：运动类型必须属于给定的类别。
- 对于不清楚的文本：列出与文本匹配的所有可能的运动类型（例如，“Football, Basketball, Volleyball”）。
- 对于明确的文本：当文本明确指出一项运动时，仅返回该特定运动类型（例如，“Football”）。
2. 输出格式：仅返回最终的字符串对象。
'''

In [6]:
def dataset_jsonl_transfer(origin_path, new_path):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []

    # 读取旧的JSONL文件
    df = pd.read_excel(origin_path)
    for _, row in df.iterrows():
        context = row["Text"]
        # genre_category = row["Genre_Cate"]
        # category = row["Category"]
        # genre_label = row["Genre"]
        label = row["Standardized_Sport"] # row["Speciality"]
        # print(label)
        message = {
            "instruction": prompt,
            "input": f"Context: {context}, Categories: {category}",
            "output": label,
        }
        messages.append(message)

    # 保存重构后的JSONL文件
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")

In [8]:
# 加载、处理数据集和测试集
train_dataset_path = "./data/classification/train_data_4k.xlsx"
train_jsonl_new_path = "./data/classification/train_4k.jsonl"
# test_dataset_path = "test_data_6k.xlsx"
# test_jsonl_new_path = "new_test_6k.jsonl"

# train_dataset_path = "train.jsonl"
# test_dataset_path = "test.jsonl"
# train_jsonl_new_path = "new_train.jsonl"
# test_jsonl_new_path = "new_test.jsonl"

if not os.path.exists(train_jsonl_new_path):
    dataset_jsonl_transfer(train_dataset_path, train_jsonl_new_path)
# if not os.path.exists(test_jsonl_new_path):
    # dataset_jsonl_transfer(test_dataset_path, test_jsonl_new_path)

In [10]:
train_df = pd.read_json(train_jsonl_new_path, lines=True)
# test_df = pd.read_json(test_jsonl_new_path, lines=True)

# train_df = pd.read_json(train_jsonl_new_path, lines=True)
# test_df = pd.read_json(test_jsonl_new_path, lines=True)

In [12]:
train_df.shape

(4044, 3)

In [14]:
# 定义主要类别列表
main_categories = ["Football", "Basketball", "Volleyball", "Table Tennis", "Tennis", "Badminton"]

# 将不属于主要类别的 'output' 转换为 'Others'
train_df['output'] = train_df['output'].apply(lambda x: x if x in main_categories else 'Others')

# 检查变换后的结果
print(train_df['output'].value_counts())


output
Others          2327
Football         574
Basketball       518
Volleyball       258
Tennis           151
Badminton        111
Table Tennis     105
Name: count, dtype: int64


In [16]:
# 假设 data 是你的数据集，包含一列 `text` 存放文本
train_df['text_length'] = train_df['instruction'].apply(len) + train_df['input'].apply(len) + train_df['output'].apply(len)

# 计算长度的分位数
max_length_95 = train_df['text_length'].quantile(0.95)
max_length_99 = train_df['text_length'].quantile(0.99)

print("95%文本的长度上限:", max_length_95)
print("99%文本的长度上限:", max_length_99)


95%文本的长度上限: 402.0
99%文本的长度上限: 412.0


In [18]:
train_df

,instruction,input,output,text_length
0,\nYou are an expert in text classification wit...,Context: WAC: OPEN WATER SWIM: W 10KM FINAL WO...,Others,401
1,\nYou are an expert in text classification wit...,Context: (L) WAC: WATER POLO: M43 WORLD AQUATI...,Others,391
2,\nYou are an expert in text classification wit...,"Context: WAC: HIGH DIVING WAC: HIGH DIVING, Ca...",Others,339
3,\nYou are an expert in text classification wit...,Context: WAC: WATER POLO: W41 WORLD AQUATICS C...,Others,387
4,\nYou are an expert in text classification wit...,"Context: RACING ON THE EDGE S4 EP03, Categorie...",Others,332
...,...,...,...,...
4039,\nYou are an expert in text classification wit...,"Context: EURO2012:REP CEKO VS POLANDIA, Catego...",Football,337
4040,\nYou are an expert in text classification wit...,Context: L/T PREMIER LEAGUE-22-23 WHU/MUN-LDN ...,Football,346
4041,\nYou are an expert in text classification wit...,Context: L/T PREMIER LEAGUE-22-23 LEI/CHE-LEI ...,Football,346
4042,\nYou are an expert in text classification wit...,Context: CAGE WARRIORS 159: SAN DIEGO CAGE WAR...,Others,395


In [20]:
from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
import torch

# 在modelscope上下载Qwen模型到本地目录下
# model_dir = snapshot_download("qwen/Qwen2.5-1.5B-Instruct", cache_dir="./", revision="master")

2024-11-11 21:04:38,565 - modelscope - INFO - PyTorch version 2.4.1+cu118 Found.
2024-11-11 21:04:38,567 - modelscope - INFO - Loading ast index from C:\Users\fhawk\.cache\modelscope\ast_indexer
2024-11-11 21:04:38,627 - modelscope - INFO - Loading done! Current index file version is 1.14.0, with md5 d7fef9503ef10c41dd0d7e9f814f7513 and a total number of 976 components indexed
D:\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./qwen/Qwen2___5-1___5B-Instruct/", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./qwen/Qwen2___5-1___5B-Instruct/", device_map="auto", torch_dtype=torch.bfloat16)
model.enable_input_require_grads()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 450
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = (
        instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    )
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = (
        [-100] * len(instruction["input_ids"])
        + response["input_ids"]
        + [tokenizer.pad_token_id]
    )
    # print("Labels:", labels)
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [23]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)

Map: 100%|████████████████████████████████████████████████████████████████| 4044/4044 [00:02<00:00, 1870.55 examples/s]


In [25]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model = get_peft_model(model, config)

In [27]:
args = TrainingArguments(
    output_dir="./output/qwen2.5-1.5B-finetuned-classification",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=1,
    num_train_epochs=1,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

In [28]:
from swanlab.integration.huggingface import SwanLabCallback
import swanlab

swanlab_callback = SwanLabCallback(
    project="Qwen2___5-finetune",
    experiment_name="Qwen2.5-1.5B-Instruct",
    description="使用通义千问Qwen2.5-1.5B-Instruct模型在Foreign体育数据集上微调-Classification",
    config={
        "model": "qwen/Qwen2.5-1.5B-Instruct",
        "dataset": "data-foreign-4k",
    },
)

In [31]:
model = model.to("cuda")

In [32]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

trainer.train()

swanlab: swanlab version 0.3.25 is available!  Upgrade: `pip install -U swanlab`
swanlab: Tracking run with swanlab version 0.3.9                                   
swanlab: Run data will be saved locally in D:\wangxiaoyang\qwen\swanlog\run-20241111_210451-a3b1799d
swanlab: 👋 Hi WXYS1209, welcome to swanlab!
swanlab: Syncing run Qwen2.5-1.5B-Instruct_Nov11_21-04-51 to the cloud
swanlab: 🌟 Run `swanlab watch -l D:\wangxiaoyang\qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@WXYS1209/Qwen2___5-finetune
swanlab: 🚀 View run at https://swanlab.cn/@WXYS1209/Qwen2___5-finetune/runs/vudgffef2dxjd84pc4ul3


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:679: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint

Step,Training Loss
1,7.427300
2,6.430500
3,4.678300
4,3.032100
5,2.041200
6,1.099700
7,0.400100
8,0.572200
9,0.148200
10,0.375500


D:\anaconda3\envs\pytorch\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in ./qwen/Qwen2___5-1___5B-Instruct/ - will assume that the vocabulary was not modified.
  warnings.warn(
D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  #

swanlab: Step 252 on key train/epoch already exists, ignored.


TrainOutput(global_step=252, training_loss=0.17836055930935785, metrics={'train_runtime': 260.2919, 'train_samples_per_second': 15.536, 'train_steps_per_second': 0.968, 'total_flos': 3810662886297600.0, 'train_loss': 0.17836055930935785, 'epoch': 0.9970326409495549})

swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the network improves
swanlab: network error, swanlab will resume uploads when the net

In [33]:
1+1

2

In [43]:
test_df = train_df.sample(n=100, random_state=42)  # random_state 设置随机种子确保每次采样一致


In [45]:
from tqdm import tqdm

In [47]:
# ====== 训练结束后的预测 ===== #

def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print(response)

    return response

model = model.to("cuda")
import time

# 记录开始时间
start_time = time.time()

test_text_list = []
result_list = []
for index, row in tqdm(test_df.iterrows(), desc=f"Predicting", unit="row"):
# for index, row in test_df.iterrows():
    instruction = row["instruction"]
    input_value = row["input"]

    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"},
    ]

    response = predict(messages, model, tokenizer)
    # messages.append({"role": "assistant", "content": f"{response}"})
    # result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
    result_list.append(response)
    test_text_list.append(input_value)
    # test_text_list.append(swanlab.Text(result_text, caption=response))

# 记录结束时间
end_time = time.time()

# 计算总用时
execution_time = end_time - start_time

# swanlab.log({"Prediction": test_text_list})
# swanlab.finish()

Predicting: 0row [00:00, ?row/s]D:\anaconda3\envs\pytorch\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\anaconda3\envs\pytorch\lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Predicting: 100row [00:13,  7.56row/s]


In [55]:
execution_time

13.23400616645813

In [52]:
test_df.shape

(962, 3)

In [46]:
train_df.shape

(4736, 3)

In [49]:
test_df['result'] = result_list

In [57]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_df.output, test_df.result)
print(f"Accuracy: {accuracy:.2f}")

In [59]:
accuracy

0.99

In [61]:
test_df.to_excel("./temp_res.xlsx", index=False)

In [47]:
test_df.output.value_counts()

output
Football                             403
Badminton                             20
Other Competitions(From 20060101)     20
Basketball                            20
Boxing                                20
Volleyball                            20
Tennis                                20
Racing                                20
Ping-Pong                             20
Billiards                             20
Ice/Water Sports(From 20060101)       20
Golf(From 20060101)                   13
Chess And Cards                        8
Bowling                                3
Name: count, dtype: int64

In [60]:
model.save_pretrained("./Qwen2-1___5B-Instruct-Finetuned")
tokenizer.save_pretrained("./Qwen2-1___5B-Instruct-Finetuned")

D:\anaconda3\envs\pytorch\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in ../qwen/Qwen2-1___5B-Instruct/ - will assume that the vocabulary was not modified.
  warnings.warn(


('./Qwen2-1___5B-Instruct-Finetuned\\tokenizer_config.json',
 './Qwen2-1___5B-Instruct-Finetuned\\special_tokens_map.json',
 './Qwen2-1___5B-Instruct-Finetuned\\vocab.json',
 './Qwen2-1___5B-Instruct-Finetuned\\merges.txt',
 './Qwen2-1___5B-Instruct-Finetuned\\added_tokens.json')